<a href="https://colab.research.google.com/github/mwl10/hetvae/blob/errors/src/catalina_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
os.chdir('/content')
! git clone --branch errors https://github.com/mwl10/hetvae
os.chdir('/content/hetvae')
! pip install -r requirements.txt
os.chdir('/content/hetvae/src')
import numpy as np
import torch
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import models
from argparse import Namespace
import torch.optim as optim
import utils
import my_utils
import pandas as pd
from glob import glob
import importlib
import vae_models
import matplotlib.pyplot as plt
import sys
from dataset import DataSet
%matplotlib inline

Cloning into 'hetvae'...
remote: Enumerating objects: 3953, done.
remote: Counting objects: 100% (989/989), done.
remote: Compressing objects: 100% (831/831), done.
remote: Total 3953 (delta 191), reused 248 (delta 158), pack-reused 2964
Receiving objects: 100% (3953/3953), 39.00 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (1136/1136), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 3.6 MB/s 
     |████████████████████████████████| 149 kB 65.0 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 15.7 MB 50.6 MB/s 
     |████████████████████████████████| 308 kB 95.9 MB/s 
     |████████████████████████████████| 112 kB 85.5 MB/s 
     |████████████████████████████████| 2.1 MB 71.5 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
     |█████████████████████████████

In [9]:
def define_model_args(trial):

    args = Namespace(
        batch_size = 8, #trial.suggest_categorical("batch_size", [8,16,32]),
        bound_variance = True,
        const_var = False,
        dataset='toy',
        dropout = trial.suggest_float("dropout", 0.0,0.5),
        elbo_weight = trial.suggest_float("elbo_weight", 0.0, 2.0),
        embed_time = trial.suggest_categorical("embed_time", [8,16,32,64,128]),
        enc_num_heads=trial.suggest_categorical("enc_num_heads", [1,2,4,8,16]),
        intensity=True,
        k_iwae=1,
        kl_annealing=False,#trial.suggest_categorical("kl_annealing",False),
        kl_zero=False, 
        latent_dim=trial.suggest_categorical("latent_dim", [8,16,32,64,128]),
        lr=trial.suggest_float("lr", 1e-7, 1e-1, log=True),
        mixing="concat_and_mix",#trial.suggest_categorical("mixing", ["concat", "concat_and_mix"]),#"separate", "interp_only", "na"]),
        mse_weight=trial.suggest_float("mse_weight",1,6),
        #n=trial.suggest_categorical("n", [8,16,32,64,128]),
        net='hetvae', 
        niters=1000, 
        norm=True, 
        normalize_input='znorm', 
        num_ref_points=trial.suggest_categorical("num_ref_points", [8,16,32,64,128]),
        rec_hidden=trial.suggest_categorical("rec_hidden", [8,16,32,64,128]),
        recon_loss=False, 
        sample_tp= 0.5, #trial.suggest_float("sample_tp", 0.1,0.9), # will be ignored
        save=True, 
        seed=0, 
        shuffle=True, 
        std=0.1, 
        var_per_dim=False, 
        width=trial.suggest_categorical("width", [8,16,32,64,128,256])
    )

    return args

In [10]:
EPOCHS = 100
FILES = glob('/content/hetvae/data/CAT/*/*')
#FILES = glob('/content/hetvae/data/CAT/*/*')[:50]


DIM = 1

In [11]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [12]:
agn_df = pd.read_csv('/content/hetvae/data/AGN_1H2106-099/1H2106-099_latest_lcs_pyroa.csv')
# handle formatting for new AGN
lcs = []
for lc_df in agn_df.groupby('Filter'):
    lc = lc_df[1][['MJD', 'Flux', 'Error']].to_numpy()
    lcs.append(lc)
# lcs[0] = lcs[0][lcs[0][:,0].argsort()]
# lcs[0] = np.concatenate((lcs[0][:10], lcs[0][410:]))
lcs = [lcs[0]]


In [13]:
def objective(trial):

    args = define_model_args(trial)

    seed = args.seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # dataset hyperparams
    num_samples=trial.suggest_int("num_samples", 1,6)
    # normalize choices for optuna
    x_by_range = trial.suggest_categorical("x_by_range", [True, False])
    y_by_range = False#trial.suggest_categorical("y_by_range", [True, False])
    #normalize_y = trial.suggest_categorical("normalize_y", ["all", "individual"])

    with suppress_stdout():
        # AGN = DataSet().add_files(FILES).files_to_numpy().handle_dups().prune_graham().resample_dataset(num_samples=num_samples) \
        #     .normalize(y_by_range=y_by_range, x_by_range=x_by_range).set_union_x().zero_fill().make_masks(frac=args.sample_tp)
        AGN = DataSet()
        AGN.dataset = lcs
        AGN = AGN.handle_dups().prune_graham().resample_dataset(num_samples=num_samples) \
             .normalize(y_by_range=y_by_range, x_by_range=x_by_range).set_union_x().zero_fill().make_masks(frac=args.sample_tp)



    LightCurves = AGN.dataset

    training, valid = np.split(LightCurves, [int(np.floor(0.8*len(LightCurves)))])# shuffle?
    
    
    train_loader = torch.utils.data.DataLoader(training, batch_size=args.batch_size)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=args.batch_size)
    
    
    net = models.load_network(args, DIM, torch.Tensor(AGN.union_x)) # , device="cuda"
    

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop"])
    optimizer = getattr(optim, optimizer_name)(net.parameters(), lr=args.lr)
    frac = trial.suggest_float("sample_tp", 0.1,0.9)
    for epoch in range(EPOCHS):
        nll_loss, mse = my_utils.train(net, optimizer, epoch, train_loader, args, device=device, frac=frac)
        #nll_loss = my_utils.evaluate(net, valid_loader, device=device)
        trial.report(nll_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return nll_loss

In [14]:
study = optuna.create_study(direction="minimize")

[I 2022-06-17 15:49:17,228] A new study created in memory with name: no-name-ebbf7a87-d4e2-43c1-8912-567a8a192240


In [15]:
study.optimize(objective, n_trials=10, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RankWarning: Polyfit may be poorly conditioned


Iter: 0, train loss: 0.1879, avg nll: 0.5021, avg kl: 0.0748, mse: 0.032299, mae: 0.149805


[I 2022-06-17 15:49:20,054] Trial 0 finished with value: 0.7559942007064819 and parameters: {'dropout': 0.32411607083877686, 'elbo_weight': 0.046403284838643755, 'embed_time': 32, 'enc_num_heads': 16, 'latent_dim': 16, 'lr': 0.05536513094249862, 'mse_weight': 5.084042462162257, 'num_ref_points': 32, 'rec_hidden': 32, 'width': 8, 'num_samples': 2, 'x_by_range': False, 'y_by_range': True, 'optimizer': 'Adam', 'sample_tp': 0.24247730278831492}. Best is trial 0 with value: 0.7559942007064819.


Iter: 0, train loss: 1.2061, avg nll: 0.5776, avg kl: 0.0457, mse: 0.044998, mae: 0.176100


[I 2022-06-17 15:49:40,638] Trial 1 finished with value: 0.5655863881111145 and parameters: {'dropout': 0.00853915766740837, 'elbo_weight': 1.6750379664612625, 'embed_time': 16, 'enc_num_heads': 8, 'latent_dim': 32, 'lr': 2.976688210164626e-07, 'mse_weight': 5.130902527037532, 'num_ref_points': 32, 'rec_hidden': 32, 'width': 128, 'num_samples': 10, 'x_by_range': True, 'y_by_range': True, 'optimizer': 'Adam', 'sample_tp': 0.30752811884049636}. Best is trial 1 with value: 0.5655863881111145.
[W 2022-06-17 15:49:40,924] Trial 2 failed because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 1.20 GiB (GPU 0; 14.76 GiB total capacity; 13.37 GiB already allocated; 113.75 MiB free; 13.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/

RuntimeError: ignored

In [ ]:
def train(net, optimizer,epoch, train_loader, args, device="cuda"):
      
      train_loss = 0.
      train_n = 0.
      avg_loglik, avg_kl, mse, mae = 0., 0., 0., 0.
      for i, train_batch in enumerate(train_loader):
          batch_len = train_batch.shape[0] 
          train_batch = train_batch.to(device)
          x = train_batch[:,:,0]
          y = train_batch[:,:,1:2]
          
          subsampled_mask = train_batch[:,:,3:4]
          recon_mask = train_batch[:,:,4:]
          sample_weight = train_batch[:,:,2:3]
          seqlen = train_batch.size(1) 
          # subsampled flux values and their corresponding masks....
          context_y = torch.cat((
              y * subsampled_mask, subsampled_mask
          ), -1) 
          recon_context_y = torch.cat((            # flux values with only recon_mask values showing
                  y * recon_mask, recon_mask
              ), -1)
          
    # #   def compute_unsupervised_loss(self, context_x, context_y, target_x, target_y, num_samples=1, beta=1):
          loss_info = net.compute_unsupervised_loss(
              x, # context_x, times
              context_y,           
              x, # target_x, same times, can project to arbitrary times 
              recon_context_y,
              num_samples=args.k_iwae,
              beta=1,
              #sample_weight = sample_weight   # default is 1. (no errors provided)

          )
          optimizer.zero_grad()
          loss_info.composite_loss.backward()
          optimizer.step()
          #scheduler.step()
          train_loss += loss_info.composite_loss.item() * batch_len
          avg_loglik += loss_info.loglik * batch_len
          avg_kl += loss_info.kl * batch_len
          mse += loss_info.mse * batch_len
          mae += loss_info.mae * batch_len
          train_n += batch_len
      
      
      if epoch % 100 == 0:
          print(
              'Iter: {}, train loss: {:.4f}, avg nll: {:.4f}, avg kl: {:.4f}, '
              'mse: {:.6f}, mae: {:.6f}'.format(
                  epoch,
                  train_loss / train_n,
                  -avg_loglik / train_n,
                  avg_kl / train_n,
                  mse / train_n,
                  mae / train_n
              )
          )
      

In [ ]:

optuna.visualization.plot_param_importances(study)

In [ ]:

optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study, params=["optimizer"])